# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../").resolve()))
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.organization_cyber_question import OrganizationCyberModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd


In [2]:
sqlstorage = SQLiteStorageProvider("organization_cyber.db")

workflow = Workflow(
    SonarQueryHandler(OrganizationCyberModel),
    sqlstorage,
    workers = 4,
    )

data = pd.read_csv("organization_names_justice.csv")
organizations = data["organization_name"].tolist()
countries = data["country"].tolist()

question_set = get_question_set(
    organizations=organizations,
    countries=countries,
)

question_set.max_questions = 0  # No limit

Loaded 0 stored question hashes from organization_cyber.db


In [3]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 193 questions with 4 workers
Processing question 1/193: Is the {organization_country} responsible for cybersecurity?

A ministry handles cybersecurity if it: Is explicitly mentioned in a national strategy/law/report as being responsible for cybersecurity policy, implementation, or technical coordination; Hosts a national CERT/CSIRT/CIRT; Leads or is a member of a cybersecurity committee, council, or working group; Oversees information security standards, network protection, or the like; Attends or participates in events, workshops, or press releases; or works with other countries or organizations on joint initiatives.
 - {'organization_country': 'Ministry of Justice in GUINEA'}
Processing question 2/193: Is the {organization_country} responsible for cybersecurity?

A ministry handles cybersecurity if it: Is explicitly mentioned in a national strategy/law/report as being responsible for cybersecurity policy, implementatio

[]

In [4]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened

,question,error,organization_country,organization,country,responsibility_level,explanation,confidence,enriched_citations,pretty_citations
0,Is the Ministry of Justice of the Russian Fede...,None,Ministry of Justice of the Russian Federation ...,Ministry of Justice of the Russian Federation,Russia,ResponsibilityLevel.LOW,The Ministry of Justice of the Russian Federat...,ConfidenceLevel.HIGH,[{'url': 'http://www.scrf.gov.ru/security/info...,[1] Doctrine of Information Security of the Ru...
1,"Is the Ministry of Justice, Communications, an...",None,"Ministry of Justice, Communications, and Forei...","Ministry of Justice, Communication and Foreign...",Tuvalu,ResponsibilityLevel.LOW,"The Ministry of Justice, Communication and For...",ConfidenceLevel.HIGH,[{'url': 'https://www.mofa.go.jp/a_o/ocn/pagei...,[1] State Minister for Foreign Affairs TAKEI V...
2,Is the Ministry of Justice in ARMENIA responsi...,None,Ministry of Justice in ARMENIA,Ministry of Justice,ARMENIA,ResponsibilityLevel.HIGH,The Ministry of Justice in Armenia is explicit...,ConfidenceLevel.HIGH,[{'url': 'https://www.dcaf.ch/sites/default/fi...,[1] [PDF] Armenia Cybersecurity Governance Ass...
3,Is the Ministry of Justice in GUINEA responsib...,None,Ministry of Justice in GUINEA,Ministry of Justice,Guinea,ResponsibilityLevel.LOW,Available information does not explicitly ment...,ConfidenceLevel.HIGH,[{'url': 'https://www.ict.gov.pg/wp-content/up...,[1] [PDF] NATIONAL CYBER SECURITY STRATEGY 202...
4,Is the Ministry of the Attorney General and Le...,None,Ministry of the Attorney General and Legal Aff...,Ministry of the Attorney General and Legal Aff...,Trinidad and Tobago,ResponsibilityLevel.LOW,The Ministry of the Attorney General and Legal...,ConfidenceLevel.HIGH,[{'url': 'https://ttcsirt.gov.tt/wp-content/up...,[1] [PDF] The Trinidad and Tobago Cyber Securi...
...,...,...,...,...,...,...,...,...,...,...
188,Is the Ministry of Popular Power for Internal ...,None,Ministry of Popular Power for Internal Relatio...,Ministry of Popular Power for Internal Relatio...,Venezuela,ResponsibilityLevel.LOW,The Ministry of Popular Power for Internal Rel...,ConfidenceLevel.MEDIUM,[{'url': 'https://bitfinance.news/en/venezuela...,[1] Venezuela creates the National Cybersecuri...
189,Is the The Royal Ministry of Justice and Publi...,None,The Royal Ministry of Justice and Public Secur...,The Royal Ministry of Justice and Public Security,Norway,ResponsibilityLevel.HIGH,The Royal Ministry of Justice and Public Secur...,ConfidenceLevel.HIGH,[{'url': 'https://www.regjeringen.no/contentas...,[1] [PDF] National Cyber Security Strategy for...
190,Is the Attorney General's Ministry in BELIZE r...,None,Attorney General's Ministry in BELIZE,Attorney General's Ministry,Belize,ResponsibilityLevel.LOW,The Attorney General’s Ministry in Belize is i...,ConfidenceLevel.HIGH,[{'url': 'https://www.pressoffice.gov.bz/minis...,[1] Ministry of National Security and Public U...
191,Is the Ministry of Justice of the Republic of ...,None,Ministry of Justice of the Republic of the Con...,Ministry of Justice of the Republic of the Congo,Congo,ResponsibilityLevel.NONE,Available information on the Democratic Republ...,ConfidenceLevel.HIGH,[{'url': 'https://dig.watch/resource/democrati...,[1] Democratic Republic of the Congo's digital...


In [5]:
wrangled = flattened.copy()


In [6]:
wrangled.enriched_citations

0      [{'url': 'http://www.scrf.gov.ru/security/info...
1      [{'url': 'https://www.mofa.go.jp/a_o/ocn/pagei...
2      [{'url': 'https://www.dcaf.ch/sites/default/fi...
3      [{'url': 'https://www.ict.gov.pg/wp-content/up...
4      [{'url': 'https://ttcsirt.gov.tt/wp-content/up...
                             ...                        
188    [{'url': 'https://bitfinance.news/en/venezuela...
189    [{'url': 'https://www.regjeringen.no/contentas...
190    [{'url': 'https://www.pressoffice.gov.bz/minis...
191    [{'url': 'https://dig.watch/resource/democrati...
192    [{'url': 'https://www.itu.int/en/ITU-D/Cyberse...
Name: enriched_citations, Length: 193, dtype: object

In [7]:
wrangled.to_excel("organization_cyber_justice.xlsx", index=False)